# Дз №2 Кластеризация текстов

In [1]:
import pandas as pd
import numpy as np

Emails = pd.read_csv("hillary-clinton-emails/Emails.csv")

Для начала выберем основные, на мой взгляд, атрибуты email сообщения. Соединяем через 'the' чтобы потом откинуть через stop_words и не получить в биграммы лишнего.

In [2]:
data = Emails[["MetadataSubject", "MetadataFrom", "MetadataTo", "ExtractedBodyText"]]
#' in '.join(map(str, list(data.xs(0))))
data['Merged'] =[(' the '.join(map(str, list(data.xs(ind))))).replace('nan', '') for ind in data.index]
data['Merged'].head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


0                 WOW the Sullivan, Jacob J the H the 
1    H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...
2     CHRIS STEVENS the Mills, Cheryl D the ;H the Thx
3    CAIRO CONDEMNATION - FINAL the Mills, Cheryl D...
4    H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...
Name: Merged, dtype: object

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(2, 2), analyzer="word", stop_words = 'english')

In [4]:
matrix = vectorizer.fit_transform(data['Merged'])
matrix.shape

(7945, 244146)

Находим частые биграммы

In [64]:
f_names = vectorizer.get_feature_names()
f_counts = np.array(matrix.sum(axis = 0))
#print matrix.shape, f_counts.shape
#print f_counts
for a,b in sorted([(f_counts[0][i], f_names[i]) for i in xrange(matrix.shape[1])])[::-1][:5]:
    print str(b)

state gov
abedin huma
mills cheryl
sullivan jacob
millscd state


In [6]:
import nltk
nltk.download()
from nltk.collocations import *

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [7]:
text = " ".join(data['Merged'])
#print text

In [8]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
tokens = nltk.wordpunct_tokenize(text)
finder = BigramCollocationFinder.from_words(tokens)
ignored_words = nltk.corpus.stopwords.words('english')
finder.apply_freq_filter(5)
finder.apply_word_filter(lambda w: len(w) < 4 or w.lower() in ignored_words)
finder.nbest(bigram_measures.pmi, 10)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


[('BHUTTO', 'ASSASSINATION'),
 ('CONSULAR', 'FEES'),
 ('Chico', 'Menashe'),
 ('Deep', 'Throat'),
 ('MARIE', 'SLAUGHTER'),
 ('BAILEY', 'HUTCHISON'),
 ('Belief', 'Blog'),
 ('Buenos', 'Aires'),
 ('DYING', 'PATIENTS'),
 ('Elliott', 'Trudeau')]

Перейдем к кластеризации

In [9]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer1 = TfidfVectorizer(ngram_range=(2, 2), analyzer="word", stop_words = 'english')
f = vectorizer1.fit_transform(data['Merged'])

In [54]:
n_clusters = 10
model = KMeans(n_clusters=n_clusters, random_state=1, n_jobs = -1)
preds = model.fit_predict(f)
print(preds[:30])

[3 5 5 5 5 5 5 5 3 5 3 5 3 5 5 5 5 5 3 3 0 5 5 0 3 5 3 5 5 2]


Посморим на размеры классов

In [55]:
for i in xrange(n_clusters):
    print "#{}: {}".format(i, len([x for x in preds if x == i]))

#0: 304
#1: 483
#2: 1295
#3: 511
#4: 95
#5: 3971
#6: 435
#7: 370
#8: 237
#9: 244


In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
clf = LogisticRegression()
print(cross_val_score(clf, f, preds).mean())

0.860284280721


In [61]:
clf.fit(f, preds)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [62]:
print ('{:^56}||{:^53}').format("best_positive", 'best_negative')
for i in range(n_clusters):
    cur_best= sorted([(coef, j) for j,coef in enumerate(clf.coef_[i])])[::-1]
    print '#{:<2}||'.format(i),
    print ('{:^16}|' * 3 + '|').format(*[f_names[j] for val, j in cur_best[:3]]),
    print ('{:^16}|' * 3).format(*[f_names[j] for val, j in cur_best[-3:]])

                     best_positive                      ||                    best_negative                    
#0 || sullivanjj state|   state gov    | sid sullivanjj ||  millscd state  | sullivan jacob | abedinh state  |
#1 ||  millscd state  |   state gov    |valmorolj state || sullivanjj state|  mills cheryl  | abedinh state  |
#2 ||   abedin huma   |  huma abedinh  |  calls abedin  ||  sullivan jacob |  mills cheryl  |   state gov    |
#3 ||  sullivan jacob |   jacob fyi    | jacob sullivan ||   mills cheryl  |   state gov    |  abedin huma   |
#4 || secretary office|   lauren ok    |     00 pm      ||   mills cheryl  |  abedin huma   |   state gov    |
#5 ||  jiloty lauren  | vermarr state  |  sid sbwhoeop  ||  sullivan jacob |   state gov    |  abedin huma   |
#6 ||   mills cheryl  |   cheryl fyi   |  update mills  ||   cheryl mills  |  abedin huma   |   state gov    |
#7 ||  abedinh state  |   state gov    |     gov ok     ||  millscd state  |  mills cheryl  |  abedin huma   |


Получили визуализацию в виде наиболее значимых признаков, отобранных логистической регрессией. По ней видно, что отрицательные коэффициенты, часто повторяются, что неудивительно. Среди положительных встречаются не только адресаты и отправители, что позволяет предположить, что кластеризация не самая тривиальная.